# Modeling workbook

In [1]:
import pandas as pd

### Data Loading & EDA

High-level data checking

In [2]:
df = pd.read_csv("data/Loan_default.csv")
df.info()
df.describe()

<class 'pandas.DataFrame'>
RangeIndex: 255347 entries, 0 to 255346
Data columns (total 18 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   LoanID          255347 non-null  str    
 1   Age             255347 non-null  int64  
 2   Income          255347 non-null  int64  
 3   LoanAmount      255347 non-null  int64  
 4   CreditScore     255347 non-null  int64  
 5   MonthsEmployed  255347 non-null  int64  
 6   NumCreditLines  255347 non-null  int64  
 7   InterestRate    255347 non-null  float64
 8   LoanTerm        255347 non-null  int64  
 9   DTIRatio        255347 non-null  float64
 10  Education       255347 non-null  str    
 11  EmploymentType  255347 non-null  str    
 12  MaritalStatus   255347 non-null  str    
 13  HasMortgage     255347 non-null  str    
 14  HasDependents   255347 non-null  str    
 15  LoanPurpose     255347 non-null  str    
 16  HasCoSigner     255347 non-null  str    
 17  Default         25534

,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Default
count,255347.000000,255347.000000,255347.000000,255347.000000,255347.000000,255347.000000,255347.000000,255347.000000,255347.000000,255347.000000
mean,43.498306,82499.304597,127578.865512,574.264346,59.541976,2.501036,13.492773,36.025894,0.500212,0.116128
std,14.990258,38963.013729,70840.706142,158.903867,34.643376,1.117018,6.636443,16.969330,0.230917,0.320379
min,18.000000,15000.000000,5000.000000,300.000000,0.000000,1.000000,2.000000,12.000000,0.100000,0.000000
25%,31.000000,48825.500000,66156.000000,437.000000,30.000000,2.000000,7.770000,24.000000,0.300000,0.000000
50%,43.000000,82466.000000,127556.000000,574.000000,60.000000,2.000000,13.460000,36.000000,0.500000,0.000000
75%,56.000000,116219.000000,188985.000000,712.000000,90.000000,3.000000,19.250000,48.000000,0.700000,0.000000
max,69.000000,149999.000000,249999.000000,849.000000,119.000000,4.000000,25.000000,60.000000,0.900000,1.000000


In [3]:
df.head()

,LoanID,Age,Income,LoanAmount,CreditScore,MonthsEmployed,NumCreditLines,InterestRate,LoanTerm,DTIRatio,Education,EmploymentType,MaritalStatus,HasMortgage,HasDependents,LoanPurpose,HasCoSigner,Default
0,I38PQUQS96,56,85994,50587,520,80,4,15.23,36,0.44,Bachelor's,Full-time,Divorced,Yes,Yes,Other,Yes,0
1,HPSK72WA7R,69,50432,124440,458,15,1,4.81,60,0.68,Master's,Full-time,Married,No,No,Other,Yes,0
2,C1OZ6DPJ8Y,46,84208,129188,451,26,3,21.17,24,0.31,Master's,Unemployed,Divorced,Yes,Yes,Auto,No,1
3,V2KKSFM3UN,32,31713,44799,743,0,3,7.07,24,0.23,High School,Full-time,Married,No,No,Business,No,0
4,EY08JDHTZP,60,20437,9139,633,8,4,6.51,48,0.73,Bachelor's,Unemployed,Divorced,No,Yes,Auto,No,0


Checking the level of class imbalance:

In [4]:
df['Default'].value_counts(normalize=True)

Default
0    0.883872
1    0.116128
Name: proportion, dtype: float64

Checking null values:

In [5]:
df.isnull().sum()

LoanID            0
Age               0
Income            0
LoanAmount        0
CreditScore       0
MonthsEmployed    0
NumCreditLines    0
InterestRate      0
LoanTerm          0
DTIRatio          0
Education         0
EmploymentType    0
MaritalStatus     0
HasMortgage       0
HasDependents     0
LoanPurpose       0
HasCoSigner       0
Default           0
dtype: int64

### Prepare dataset to train the model

Separate features and target ("Default" column is the target):

In [6]:
X = df.drop("Default", axis=1)
y = df["Default"]


Split with stratification (keeping the same class proportion within test and train data):

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)


Keep IDs out of modeling to avoid IDs break the model:

In [8]:
id_col = 'LoanID'

X_train_ids = X_train[id_col].copy()
X_test_ids = X_test[id_col].copy()

X_train = X_train.drop(columns=[id_col])
X_test = X_test.drop(columns=[id_col])


Identify categorical and numerical features

In [9]:
categorical_features = X_train.select_dtypes(include=["object", "string", "category"]).columns.tolist()
numerical_features = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()

print(categorical_features)
print(numerical_features)

['Education', 'EmploymentType', 'MaritalStatus', 'HasMortgage', 'HasDependents', 'LoanPurpose', 'HasCoSigner']
['Age', 'Income', 'LoanAmount', 'CreditScore', 'MonthsEmployed', 'NumCreditLines', 'InterestRate', 'LoanTerm', 'DTIRatio']


Standardization for numeric fields & one hot encoding for categorical fields

In [10]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Preprocessing (fit will happen inside the Pipeline to avoid leakage/duplication)
# drop="if_binary" keeps one dummy for 2-level categories (e.g. Yes/No -> 0/1-style column)
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(handle_unknown="ignore", drop="if_binary"), categorical_features),
    ],
    remainder="drop"
)


### Train and use the model

Train a Logistic Regression model:

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

# Train a single Pipeline that contains BOTH preprocessing and the model
pipeline = Pipeline(steps=[
    ("prep", preprocessor),
    ("model", LogisticRegression(max_iter=1000, class_weight="balanced"))
])

pipeline.fit(X_train, y_train)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('prep', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains s

Convert back to DataFrame with feature names (Helps the analysis later):

In [12]:
# Note: preprocessing is already fitted inside `pipeline`
X_train_transformed = pipeline.named_steps["prep"].transform(X_train)
X_test_transformed = pipeline.named_steps["prep"].transform(X_test)

feature_names = pipeline.named_steps["prep"].get_feature_names_out()

X_train_df = pd.DataFrame(X_train_transformed, columns=feature_names, index=X_train.index)
X_test_df = pd.DataFrame(X_test_transformed, columns=feature_names, index=X_test.index)

X_train_df.head()


,num__Age,num__Income,num__LoanAmount,num__CreditScore,num__MonthsEmployed,num__NumCreditLines,num__InterestRate,num__LoanTerm,num__DTIRatio,cat__Education_Bachelor's,...,cat__MaritalStatus_Married,cat__MaritalStatus_Single,cat__HasMortgage_Yes,cat__HasDependents_Yes,cat__LoanPurpose_Auto,cat__LoanPurpose_Business,cat__LoanPurpose_Education,cat__LoanPurpose_Home,cat__LoanPurpose_Other,cat__HasCoSigner_Yes
15826,0.099486,-1.167307,1.698627,0.312274,-1.170591,-0.449335,-1.335928,1.414459,1.512942,0.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
147371,0.299620,1.319747,-0.864170,-0.505800,1.714171,0.445947,0.185568,0.707489,-0.045473,0.0,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0
178180,0.232908,0.453497,-1.700954,0.903803,1.396847,0.445947,-1.201856,-0.706451,1.123338,0.0,...,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
126915,-0.100649,-1.191966,-1.432895,-1.449730,-1.661000,0.445947,0.723364,0.000519,1.123338,1.0,...,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
163930,-1.568299,0.434508,1.707671,-1.613345,0.416028,0.445947,0.898110,1.414459,-0.218630,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


Get the probability of default (PD) results for test

In [13]:
# Probability of Default (PD) for the test set
pd_test = pipeline.predict_proba(X_test)[:, 1]


### Save trained pipeline

This saves preprocessing + model together so you can reuse it in a separate scoring notebook.


In [14]:
import joblib

bundle = {
    "pipeline": pipeline,
    "id_col": id_col,
    "target_col": "Default"
}

joblib.dump(bundle, "pd_pipeline.joblib")


['pd_pipeline.joblib']

### Evaluate model performance

Evaluate ranking quality (ROC-AUC):

In [15]:
from sklearn.metrics import roc_auc_score

roc_auc = roc_auc_score(y_test, pd_test)
print("ROC-AUC: ", roc_auc)


ROC-AUC:  0.7531772341541364


Creating evaluation table:

In [16]:
results = pd.DataFrame({
    "actual_defaults": y_test.values,
    "pd": pd_test,
})

results.head()

,actual_defaults,pd
0,0,0.186289
1,0,0.139964
2,0,0.378479
3,0,0.569522
4,0,0.407866


Creating deciles for risk buckets:

In [17]:
results["decile"] = pd.qcut(results["pd"], 10, labels=False)
results.head()

,actual_defaults,pd,decile
0,0,0.186289,1
1,0,0.139964,0
2,0,0.378479,4
3,0,0.569522,7
4,0,0.407866,4


Chech default rate by decile:

In [18]:
decile_summary = results.groupby("decile")["actual_defaults"].mean()
print(decile_summary)

decile
0    0.017231
1    0.032700
2    0.047190
3    0.057764
4    0.070687
5    0.087527
6    0.114549
7    0.155864
8    0.213237
9    0.364598
Name: actual_defaults, dtype: float64


Prepare final scored dataset

In [19]:
scored_test = pd.DataFrame({
    "customer_id": X_test_ids.values,
    "actual_defaults": y_test.values,
    "pd": pd_test
})

scored_test.head()

,customer_id,actual_defaults,pd
0,SJJNN2DFZ7,0,0.186289
1,GOC4OI55T8,0,0.139964
2,QO6XNPEH8P,0,0.378479
3,3VLC51ZYU8,0,0.569522
4,OUGXQD95MI,0,0.407866
